In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ABIDE

Mounted at /content/drive
/content/drive/MyDrive/ABIDE


In [2]:
!pip install nilearn

from nilearn.image import load_img
from nilearn.datasets import fetch_atlas_schaefer_2018
from nilearn.input_data import NiftiLabelsMasker
import numpy as np
from scipy.spatial.distance import squareform
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# Fetch the atlas with 100 regions of interest (ROIs)
atlas = fetch_atlas_schaefer_2018(n_rois=100)
masker = NiftiLabelsMasker(labels_img=atlas.maps, standardize=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 100.5 MB/s eta 0:00:00


[_add_readme_to_default_data_locations] Added README.md to /root/nilearn_data

[get_dataset_dir] Dataset created in /root/nilearn_data/schaefer_2018

[fetch_single_file] Downloading data from 
https://raw.githubusercontent.com/ThomasYeoLab/CBIG/v0.14.3-Update_Yeo2011_Schaefer2018_labelname/stable_projects/b
rain_parcellation/Schaefer2018_LocalGlobal/Parcellations/MNI/Schaefer2018_100Parcels_7Networks_order.txt ...

[fetch_single_file]  ...done. (0 seconds, 0 min)

[fetch_single_file] Downloading data from 
https://raw.githubusercontent.com/ThomasYeoLab/CBIG/v0.14.3-Update_Yeo2011_Schaefer2018_labelname/stable_projects/b
rain_parcellation/Schaefer2018_LocalGlobal/Parcellations/MNI/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_1mm.
nii.gz ...

[fetch_single_file]  ...done. (0 seconds, 0 min)

In [3]:
imaging_files = os.listdir("func_preproc")
print(f"Number of imaging files: {len(imaging_files)}")
print("First 10 imaging files:", imaging_files[:10])


Number of imaging files: 247
First 10 imaging files: ['Pitt_0050014_func_preproc.nii.gz', 'Pitt_0050025_func_preproc.nii.gz', 'Pitt_0050041_func_preproc.nii.gz', 'Pitt_0050042_func_preproc.nii.gz', 'Pitt_0050045_func_preproc.nii.gz', 'Pitt_0050047_func_preproc.nii.gz', 'Pitt_0050052_func_preproc.nii.gz', 'Olin_0050104_func_preproc.nii.gz', 'Olin_0050107_func_preproc.nii.gz', 'Olin_0050112_func_preproc.nii.gz']


In [4]:
import pandas as pd

# Function to extract subject ID from the filename
def get_id(name):
    parts = name.split('_')
    for part in parts:
        if part.isdigit():
            return part.lstrip('0')  # Strip leading zeros
    return None


# Load the phenotypic data
phenotypic_data = pd.read_csv('Updated_Phenotypic_V1_0b_preprocessed1_corrected_version2.csv')
phenotypic_data['SUB_ID'] = phenotypic_data['SUB_ID'].astype(str).str.strip()

# List of fMRI files
imaging_files = os.listdir("func_preproc")

# Extract IDs from imaging files
image_ids = [get_id(f) for f in imaging_files]
print(f"First 10 extracted image IDs: {image_ids[:10]}")

# Ensure there are no None values
image_ids = [id_ for id_ in image_ids if id_ is not None]

# Check the first few `SUB_ID`s in phenotypic data
print("First 10 SUB_IDs in phenotypic data:", phenotypic_data['WISC_IV_PSI'])

First 10 extracted image IDs: ['50014', '50025', '50041', '50042', '50045', '50047', '50052', '50104', '50107', '50112']
First 10 SUB_IDs in phenotypic data: 0        89.0
1       100.0
2       102.0
3       117.0
4        89.0
        ...  
1107    120.0
1108    105.0
1109    108.0
1110    116.0
1111     87.0
Name: WISC_IV_PSI, Length: 1112, dtype: float64


In [19]:
from tqdm import tqdm

# #WIP
# def standardise_column(column):
#   print(column)
#   return (column - column.mean())/column.std()

def standardise_all(variables, phenotypic_data):
  print(variables)
  for variable in variables:
    print(type(phenotypic_data[variable]))
    print(phenotypic_data[variable])
    print(phenotypic_data["WISC_IV_PSI"])

    phenotypic_data[variable] = (phenotypic_data[variable] - phenotypic_data[variable].mean())/phenotypic_data[variable].std()
    print(type(phenotypic_data[variable]))
    print(phenotypic_data[variable])
    print(phenotypic_data["WISC_IV_PSI"])
  return phenotypic_data

def get_personality_score(personality_type, personality_dictionary, row):
  pos = np.mean([row[variable] for variable in personality_dictionary[personality_type]["Pos"]])
  neg = np.mean([row[variable] for variable in personality_dictionary[personality_type]["Neg"]])
  return pos - neg

def get_personality(personality_dictionary, row):
  choleric_score = get_personality_score("choleric", personality_dictionary, row)
  print(choleric_score)
  sanguine_score = get_personality_score("sanguine", personality_dictionary, row)
  print(sanguine_score)
  phlegmatic_score = get_personality_score("phlegmatic", personality_dictionary, row)
  print(phlegmatic_score)
  melancholic_score = get_personality_score("melancholic", personality_dictionary, row)
  print(melancholic_score)
  if max(choleric_score, sanguine_score, phlegmatic_score, melancholic_score) == choleric_score:
    return "choleric"
  if max(choleric_score, sanguine_score, phlegmatic_score, melancholic_score) == sanguine_score:
    return "sanguine"
  if max(choleric_score, sanguine_score, phlegmatic_score, melancholic_score) == phlegmatic_score:
    return "phlegmatic"
  if max(choleric_score, sanguine_score, phlegmatic_score, melancholic_score) == melancholic_score:
    return "melancholic"

# Scores for receptive communication, expressive communication, written communication, interpersonal skills, coping skills, processing speed
personality_variables = ["VINELAND_RECEPTIVE_V_SCALED", "VINELAND_EXPRESSIVE_V_SCALED",
                        "VINELAND_WRITTEN_V_SCALED", "VINELAND_INTERPERSONAL_V_SCALED",
                        "VINELAND_COPING_V_SCALED", "WISC_IV_PSI"]

phenotypic_data = standardise_all(personality_variables, phenotypic_data)

print(phenotypic_data["WISC_IV_PSI"])

personality_dictionary = {
  "choleric": {"Pos":["WISC_IV_PSI"], "Neg": ["VINELAND_RECEPTIVE_V_SCALED", "VINELAND_INTERPERSONAL_V_SCALED", "VINELAND_COPING_V_SCALED"]},
  "sanguine": {"Pos":["VINELAND_EXPRESSIVE_V_SCALED", "VINELAND_INTERPERSONAL_V_SCALED"], "Neg": ["VINELAND_WRITTEN_V_SCALED"]},
  "phlegmatic": {"Pos":["VINELAND_RECEPTIVE_V_SCALED", "VINELAND_COPING_V_SCALED"], "Neg": ["WISC_IV_PSI"]},
  "melancholic": {"Pos":["VINELAND_WRITTEN_V_SCALED"], "Neg": ["VINELAND_EXPRESSIVE_V_SCALED"]}
}

def add_personalities(personality_dictionary, phenotypic_data):
  for idx, row in tqdm(phenotypic_data.iterrows(), total=phenotypic_data.shape[0]):
    phenotypic_data.loc[idx, "PERSONALITY"] = get_personality(personality_dictionary, row)
  return phenotypic_data

phenotypic_data = add_personalities(personality_dictionary, phenotypic_data)

phenotypic_data.to_csv("with_personalities.csv", encoding='utf-8')

['VINELAND_RECEPTIVE_V_SCALED', 'VINELAND_EXPRESSIVE_V_SCALED', 'VINELAND_WRITTEN_V_SCALED', 'VINELAND_INTERPERSONAL_V_SCALED', 'VINELAND_COPING_V_SCALED', 'WISC_IV_PSI']
<class 'pandas.core.series.Series'>
0       0.805293
1      -0.347941
2      -0.347941
3       0.228676
4      -0.492095
          ...   
1107    0.228676
1108   -0.924558
1109   -1.357020
1110   -0.492095
1111   -1.645329
Name: VINELAND_RECEPTIVE_V_SCALED, Length: 1112, dtype: float64
0      -0.907366
1      -0.204103
2      -0.076237
3       0.882759
4      -0.907366
          ...   
1107    1.074558
1108    0.115562
1109    0.307361
1110    0.818826
1111   -1.035232
Name: WISC_IV_PSI, Length: 1112, dtype: float64
<class 'pandas.core.series.Series'>
0       0.805293
1      -0.347941
2      -0.347941
3       0.228676
4      -0.492095
          ...   
1107    0.228676
1108   -0.924558
1109   -1.357020
1110   -0.492095
1111   -1.645329
Name: VINELAND_RECEPTIVE_V_SCALED, Length: 1112, dtype: float64
0      -0.907366
1  

  7%|▋         | 81/1112 [00:00<00:01, 807.76it/s]

-1.2591912910281584
-1.1141955954891836
1.5739212905745439
1.0109110166948234
0.36419387007178095
-0.4652303427086931
0.22129275078591049
-0.87918028512929
0.1706643900081638
-1.6514514141917287
-0.5613073179882095
2.172880229049204
0.4637375765108068
1.2328649670601972
-0.359015501486044
-1.4570497566623124
-0.3417403570913806
-1.9854499951375286
0.1977448896693771
2.1737702675189663
1.4334025035800737
-0.8575484196861443
-1.9439502812547302
1.8892857241012662
-0.6402946612354616
-0.4752858033013081
0.6948117839962871
1.0198114013924455
1.1922371751309788
1.534815670860798
-0.8513847290951111
-2.4732165434287943
2.18313037119632
-0.5991718761574953
-1.6293185636586256
-1.028542864596238
1.1642371895267016
-3.1711663463770603
-0.6885022745156395
2.9036724339282234
2.472573377373963
-0.7523596469287228
-2.8089982201866954
0.7219762809283121
-0.18637736055965243
-2.2758997704183335
0.838081593906227
2.3249129239254387
-0.6527945848434724
-1.7925386100055862
1.2275972138915971
1.445648178

 21%|██        | 235/1112 [00:00<00:01, 681.53it/s]

-0.3985680331496361
1.9675077178504536
0.27784372897286086
-2.32919419478042
-1.230551365819681
1.5345506334355676
1.2061535985458378
-2.187841961541332
1.473879076555845
-1.0143470535503352
-1.224612143121622
0.7237563578678365
-0.2156847556224989
-0.04170371784189063
0.2713771666115071
1.0171412859831586
1.338102966057501
-0.13864246155317905
-1.3086465938629868
-1.1725652132446127
2.0380576730661186
-0.4530790585374084
-1.972409161665456
0.14588688633481395
1.7002568020641888
0.41343475672993757
-1.1927068264493095
-1.7495446463078728
1.9236880394260563
-2.3185585376135665
-1.5796604579948486
2.0279678419310674
1.6139523554268118
-1.5350267503397421
-1.1124855946040322
0.42592123740370313
-0.1759005358658845
1.5292104026169944
0.10367504776327849
-1.3076871771953649
1.421132379358318
0.34411905242199414
-1.4728712389332705
-0.43910289295630633
-0.04767363112436751
1.634399175374416
0.1255453682617359
-2.474996620368319
1.3284368644163298
1.1700445694658201
-1.641366845568699
-1.4588

...

-0.7512045710337298
-1.8665358229328706
0.2899092349831884
0.0016669791541733536
0.14468700596952827
0.07835829100323422
-0.1501681571897951
-0.19130738673326336
0.14584208186452108
0.9215359094367417
-0.5822352031349187
2.846368192008323
-2.8371677654312606
-2.7865791969846248
2.902782395458461
1.5619187031260133
-0.3129072558002707
-0.9851996780420463
-0.7315977826018667
-0.4581171754965424
1.0612702882223923
0.5321426819931648
-1.4561597181925503
-1.459757964461137
1.0677909495429159
1.6976698108403872
-1.0169723644893292
2.132045919944355
-1.7581805810704014
-1.8256929154939292
1.5967908344557058
1.8010054599510505
-2.808440359346837
-1.036290934231192
2.4680452341040504
-0.43172413210687416
-2.5446728757857002
0.8518987714598109
2.172880229049204
1.0491617640984208
0.8205995878949173
-0.4519561104868005
-2.1816116922529964
-0.31745554854542335
-0.5659435685103462
0.17087258984217124
1.30607602174967
1.259445234353957
-2.591516797904513
-0.8313143417565227
2.3240228854556766
0.6106

 96%|█████████▋| 1073/1112 [00:01<00:00, 783.32it/s]

-1.7811755959072841
-0.3807079205939202
1.8498457790097187
0.5708336245218399
1.2117163438184535
2.0902133602046904
-1.9215312580526598
-1.600182066840925
0.011298468741963497
-1.4484338592532566
-0.6349399899212189
3.0485848210463606


100%|██████████| 1112/1112 [00:01<00:00, 728.91it/s]


-0.764711184016436
0.47957562538509024
0.31045157339781687
-0.29775065971721837
-1.1325179887609196
-1.1660197848072689
0.586589414409341
2.4662651571645267
0.03639506798847841
1.0138962912531182
-0.5594164247937845
-0.7342678980111533
-1.6829410150339603
1.5132339271414301
0.9499951848383464
-1.4606099105413615
0.6911187391509763
0.5009176862861852
-1.2857698279918592
0.7264264732771231
0.5074051122151934
0.8268805663971679
-0.6214585574857774
0.29435942733199894
0.13147082260648485
2.0128366004549725
-0.3055752144425374
-2.472326504959032
0.9435373161209405
0.9451106617956362
-1.159917061955242
0.005424691565488304
-0.7659767283131138
-0.21536886565140811
0.8462256968856567
-0.1546183495386062
0.426573293483921
-0.688024209820052
-0.8518424717641679
0.7157460116399768
2.158584863417633
-0.4717256494222593
-2.320746524084621
0.43304154516180077
0.4126988716065005
1.065745835178161
-0.8016356633967779
-0.43821285448654423
-1.5260590280813349
1.0918031258532976
1.5535155531086917
-0.432

In [ ]:
import os

fmri_directory = '/content/drive/MyDrive/ABIDE/func_preproc'
fmri_files = os.listdir(fmri_directory)
print(f"Files in '{fmri_directory}':")
print(fmri_files)


Files in '/content/drive/MyDrive/ABIDE/func_preproc':
['Pitt_0050014_func_preproc.nii.gz', 'Pitt_0050025_func_preproc.nii.gz', 'Pitt_0050041_func_preproc.nii.gz', 'Pitt_0050042_func_preproc.nii.gz', 'Pitt_0050045_func_preproc.nii.gz', 'Pitt_0050047_func_preproc.nii.gz', 'Pitt_0050052_func_preproc.nii.gz', 'Olin_0050104_func_preproc.nii.gz', 'Olin_0050107_func_preproc.nii.gz', 'Olin_0050112_func_preproc.nii.gz', 'Olin_0050118_func_preproc.nii.gz', 'Olin_0050125_func_preproc.nii.gz', 'Olin_0050133_func_preproc.nii.gz', 'OHSU_0050143_func_preproc.nii.gz', 'OHSU_0050144_func_preproc.nii.gz', 'OHSU_0050153_func_preproc.nii.gz', 'OHSU_0050166_func_preproc.nii.gz', 'OHSU_0050168_func_preproc.nii.gz', 'OHSU_0050169_func_preproc.nii.gz', 'OHSU_0050170_func_preproc.nii.gz', 'SDSU_0050182_func_preproc.nii.gz', 'SDSU_0050187_func_preproc.nii.gz', 'SDSU_0050188_func_preproc.nii.gz', 'SDSU_0050190_func_preproc.nii.gz', 'SDSU_0050191_func_preproc.nii.gz', 'SDSU_0050194_func_preproc.nii.gz', 'SDSU_005

In [ ]:
file_to_check = 'Pitt_0050041_func_preproc.nii.gz'
if file_to_check in fmri_files:
    print(f"File '{file_to_check}' exists in '{fmri_directory}'.")
else:
    print(f"File '{file_to_check}' does NOT exist in '{fmri_directory}'.")


File 'Pitt_0050041_func_preproc.nii.gz' exists in '/content/drive/MyDrive/ABIDE/func_preproc'.


In [ ]:
# Match phenotypic data with imaging data
matched_data = phenotypic_data[phenotypic_data['SUB_ID'].isin(image_ids)].reset_index(drop=True)
print(f"Number of matched subjects: {len(matched_data)}")


Number of matched subjects: 172


In [ ]:
import os

# Change current working directory
os.chdir('/content/drive/MyDrive/ABIDE')
print(f"Current working directory: {os.getcwd()}")


Current working directory: /content/drive/MyDrive/ABIDE


In [ ]:
from nilearn.image import load_img
from nilearn.datasets import fetch_atlas_schaefer_2018
from nilearn.input_data import NiftiLabelsMasker
import numpy as np
from scipy.spatial.distance import squareform
from tqdm import tqdm
import os
import shutil

# Fetch the atlas
atlas = fetch_atlas_schaefer_2018(n_rois=100)
masker = NiftiLabelsMasker(labels_img=atlas.maps, standardize=True)

# Copy fMRI files from Google Drive to Colab's local directory for more reliable access
fmri_directory_drive = "/content/drive/MyDrive/ABIDE/func_preproc"
local_directory = "/content/func_preproc"
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

imaging_files = os.listdir(fmri_directory_drive)
for fmri_file in imaging_files:
    shutil.copy(f"{fmri_directory_drive}/{fmri_file}", local_directory)

# Initialize feature and label lists
features_list = []
labels_list = []

# Loop through matched data to extract features
for idx, row in tqdm(matched_data.iterrows(), total=matched_data.shape[0]):
    sub_id = row['SUB_ID']
    fmri_file = next((f for f in imaging_files if sub_id in f), None)
    if fmri_file:
        fmri_file_path = f"/content/func_preproc/{fmri_file}"
        if os.path.exists(fmri_file_path):
            try:
                # Load the fMRI image
                fmri_img = load_img(fmri_file_path)
                # Extract time series data using the atlas-based masker
                time_series = masker.fit_transform(fmri_img)

                # Replace NaNs/Infs in time series
                time_series = np.nan_to_num(time_series, nan=0.0, posinf=0.0, neginf=0.0)

                # Compute the correlation matrix
                correlation_matrix = np.corrcoef(time_series.T)

                # Replace NaNs/Infs in correlation matrix
                correlation_matrix = np.nan_to_num(correlation_matrix, nan=0.0, posinf=0.0, neginf=0.0)

                # Flatten the upper triangle of the correlation matrix to use as features
                features = squareform(correlation_matrix, checks=False)
                features_list.append(features)
                labels_list.append(row['DX_GROUP'])
            except (OSError, ConnectionAbortedError) as e:
                print(f"Error loading file '{fmri_file_path}': {e}")
        else:
            print(f"File '{fmri_file_path}' does not exist.")
    else:
        print(f"No fMRI file found for subject {sub_id}")



[get_dataset_dir] Dataset found in /root/nilearn_data/schaefer_2018

 41%|████▏     | 71/172 [04:28<06:38,  3.94s/it]/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
100%|██████████| 172/172 [10:16<00:00,  3.58s/it]


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assuming feature extraction is complete and features_list and labels_list are populated

# Convert features_list to a NumPy array
X = np.array(features_list)

# Convert labels_list to a NumPy array
y = np.array(labels_list)

# Encode labels: 1 -> 1 (ASD), 2 -> 0 (Control)
y_encoded = (y == 1).astype(int)

# Optional: Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")


Training set size: 137 samples
Testing set size: 35 samples


In [ ]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import layers, models, regularizers, callbacks
from tensorflow.keras.optimizers import Adam,RMSprop
from sklearn.utils import class_weight
import numpy as np

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import layers, models, regularizers, callbacks
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.utils import class_weight
import numpy as np

# Initialize cross-validation with StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

# Define the learning rate schedule function
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr + 0.0001  # Gradually increase during warm-up
    return lr * 0.95  # Gradually decrease after epoch 10

for train_idx, test_idx in kfold.split(X_scaled, y_encoded):
    X_train_fold, X_test_fold = X_scaled[train_idx], X_scaled[test_idx]
    y_train_fold, y_test_fold = y_encoded[train_idx], y_encoded[test_idx]

    # Compute class weights for the current fold
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_fold), y=y_train_fold)
    class_weights = dict(enumerate(class_weights))

    # Define a more complex model with LeakyReLU activations and increased number of units
    model = models.Sequential([
        layers.Input(shape=(X_train_fold.shape[1],)),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.1),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(128),
        layers.LeakyReLU(alpha=0.1),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(64),
        layers.LeakyReLU(alpha=0.1),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(32),
        layers.LeakyReLU(alpha=0.1),
        layers.BatchNormalization(),
        layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model with RMSprop optimizer
    model.compile(optimizer=RMSprop(learning_rate=0.0005),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Early stopping callback with increased patience
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

    # Learning rate scheduler callback
    lr_scheduler_callback = callbacks.LearningRateScheduler(lr_schedule, verbose=1)

    # Train the model
    history = model.fit(X_train_fold, y_train_fold,
                        epochs=150,
                        batch_size=32,
                        validation_data=(X_test_fold, y_test_fold),
                        class_weight=class_weights,
                        callbacks=[early_stopping, lr_scheduler_callback],
                        verbose=0)

    # Evaluate the model on the validation set
    score = model.evaluate(X_test_fold, y_test_fold, verbose=0)
    fold_accuracies.append(score[1])
    print(f"Fold Test Accuracy: {score[1] * 100:.2f}%")

# Print the average cross-validation accuracy
print(f"Average Cross-Validation Accuracy: {np.mean(fold_accuracies) * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Epoch 1: LearningRateScheduler setting learning rate to 0.0006000000237487257.

Epoch 2: LearningRateScheduler setting learning rate to 0.0007000000284984708.

Epoch 3: LearningRateScheduler setting learning rate to 0.000800000033248216.

Epoch 4: LearningRateScheduler setting learning rate to 0.0009000000379979611.

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000427477062.

Epoch 6: LearningRateScheduler setting learning rate to 0.0011000000474974514.

Epoch 7: LearningRateScheduler setting learning rate to 0.0011999999940395356.

Epoch 8: LearningRateScheduler setting learning rate to 0.0012999999405816198.

Epoch 9: LearningRateScheduler setting learning rate to 0.001399999887123704.
Fold Test Accuracy: 51.43%

Epoch 1: LearningRateScheduler setting learning rate to 0.0006000000237487257.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Epoch 2: LearningRateScheduler setting learning rate to 0.0007000000284984708.

Epoch 3: LearningRateScheduler setting learning rate to 0.000800000033248216.

Epoch 4: LearningRateScheduler setting learning rate to 0.0009000000379979611.

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000427477062.

Epoch 6: LearningRateScheduler setting learning rate to 0.0011000000474974514.

Epoch 7: LearningRateScheduler setting learning rate to 0.0011999999940395356.

Epoch 8: LearningRateScheduler setting learning rate to 0.0012999999405816198.

Epoch 9: LearningRateScheduler setting learning rate to 0.001399999887123704.
Fold Test Accuracy: 60.00%

Epoch 1: LearningRateScheduler setting learning rate to 0.0006000000237487257.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Epoch 2: LearningRateScheduler setting learning rate to 0.0007000000284984708.

Epoch 3: LearningRateScheduler setting learning rate to 0.000800000033248216.

Epoch 4: LearningRateScheduler setting learning rate to 0.0009000000379979611.

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000427477062.

Epoch 6: LearningRateScheduler setting learning rate to 0.0011000000474974514.

Epoch 7: LearningRateScheduler setting learning rate to 0.0011999999940395356.

Epoch 8: LearningRateScheduler setting learning rate to 0.0012999999405816198.

Epoch 9: LearningRateScheduler setting learning rate to 0.001399999887123704.

Epoch 10: LearningRateScheduler setting learning rate to 0.0014999998336657882.
Fold Test Accuracy: 70.59%

Epoch 1: LearningRateScheduler setting learning rate to 0.0006000000237487257.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Epoch 2: LearningRateScheduler setting learning rate to 0.0007000000284984708.

Epoch 3: LearningRateScheduler setting learning rate to 0.000800000033248216.

Epoch 4: LearningRateScheduler setting learning rate to 0.0009000000379979611.

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000427477062.

Epoch 6: LearningRateScheduler setting learning rate to 0.0011000000474974514.

Epoch 7: LearningRateScheduler setting learning rate to 0.0011999999940395356.

Epoch 8: LearningRateScheduler setting learning rate to 0.0012999999405816198.

Epoch 9: LearningRateScheduler setting learning rate to 0.001399999887123704.

Epoch 10: LearningRateScheduler setting learning rate to 0.0014999998336657882.

Epoch 11: LearningRateScheduler setting learning rate to 0.0014249997911974788.

Epoch 12: LearningRateScheduler setting learning rate to 0.0013537498016376048.

Epoch 13: LearningRateScheduler setting learning rate to 0.0012860622839070857.

Epoch 14: LearningRateScheduler setti

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Epoch 1: LearningRateScheduler setting learning rate to 0.0006000000237487257.

Epoch 2: LearningRateScheduler setting learning rate to 0.0007000000284984708.

Epoch 3: LearningRateScheduler setting learning rate to 0.000800000033248216.

Epoch 4: LearningRateScheduler setting learning rate to 0.0009000000379979611.

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000427477062.

Epoch 6: LearningRateScheduler setting learning rate to 0.0011000000474974514.

Epoch 7: LearningRateScheduler setting learning rate to 0.0011999999940395356.

Epoch 8: LearningRateScheduler setting learning rate to 0.0012999999405816198.
Fold Test Accuracy: 35.29%
Average Cross-Validation Accuracy: 58.76%


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7937 - loss: 0.5151 
Test Accuracy: 80.00%
